In [24]:

# import h5py
import time
# import threading
# from collections import OrderedDict

from ophyd import Component, Device, DeviceStatus, Signal
from ophyd import FormattedComponent
from ophyd import EpicsMotor, EpicsScaler
from ophyd.scaler import ScalerCH
from ophyd import EpicsSignal, EpicsSignalRO, EpicsSignalWithRBV
from ophyd import PVPositioner, PVPositionerPC
from ophyd import AreaDetector, PcoDetectorCam
from ophyd import SingleTrigger, ImagePlugin, HDF5Plugin, TIFFPlugin, ProcessPlugin
from ophyd import ADComponent
from ophyd.areadetector.filestore_mixins import FileStoreHDF5IterativeWrite
from ophyd.areadetector.plugins import PluginBase
import ophyd.status  # for the wait() function

from bluesky import IllegalMessageSequence
import bluesky.suspenders

# bps.mv, bps.mvr, ... (make it obvious where this originate)
import bluesky.plan_stubs as bps
import bluesky.plans as bp
import bluesky.preprocessors as bpp
from bluesky.simulators import summarize_plan

import apstools.devices as APS_devices
import apstools.plans as APS_plans

In [2]:
# simulated devices

A_shutter = APS_devices.SimulatedApsPssShutterWithStatus(name="A_shutter")

In [14]:
A_shutter.summary()

data keys (* hints)
-------------------
 A_shutter_busy
 A_shutter_close_signal
 A_shutter_open_signal
 A_shutter_pss_state

read attrs
----------
busy                 Signal              ('A_shutter_busy')
open_signal          Signal              ('A_shutter_open_signal')
close_signal         Signal              ('A_shutter_close_signal')
pss_state            Signal              ('A_shutter_pss_state')

config keys
-----------

configuration attrs
-------------------

unused attrs
------------



In [7]:
A_shutter.isOpen

# check ideas 108

True

# Operation shutter in 6BM_A

In [15]:
# 6bm shutter
A_shutter = APS_devices.ApsPssShutterWithStatus(
    "6bmb1:rShtrA",
    "PA:06BM:STA_A_FES_OPEN_PL",
    name="A_shutter",
    )

# Configuring AreaDetector

In [ ]:
from ophyd import PointGreyDetectorCam, ProcessPlugin

# -- user config



# -- operation
ADPV_prefix = "1idPG3"  # new detector
USER6BMA_ROOT_DIR = "TODO"
FILE_PATH = "TODO"


class MyPointGreyDetectorCam(PointGreyDetectorCam):
    """PointGrey Grasshopper3 cam plugin customizations (properties)"""
    auto_exposure_on_off = ADComponent(EpicsSignalWithRBV, "AutoExposureOnOff")
    auto_exposure_auto_mode = ADComponent(EpicsSignalWithRBV, "AutoExposureAutoMode")
    sharpness_on_off = ADComponent(EpicsSignalWithRBV, "SharpnessOnOff")
    sharpness_auto_mode = ADComponent(EpicsSignalWithRBV, "SharpnessAutoMode")
    gamma_on_off = ADComponent(EpicsSignalWithRBV, "GammaOnOff")
    shutter_auto_mode = ADComponent(EpicsSignalWithRBV, "ShutterAutoMode")
    gain_auto_mode = ADComponent(EpicsSignalWithRBV, "GainAutoMode")
    trigger_mode_on_off = ADComponent(EpicsSignalWithRBV, "TriggerModeOnOff")
    trigger_mode_auto_mode = ADComponent(EpicsSignalWithRBV, "TriggerModeAutoMode")
    trigger_delay_on_off = ADComponent(EpicsSignalWithRBV, "TriggerDelayOnOff")
    frame_rate_on_off = ADComponent(EpicsSignalWithRBV, "FrameRateOnOff")
    frame_rate_auto_mode = ADComponent(EpicsSignalWithRBV, "FrameRateAutoMode")
    # there are other PG3 properties, ignore them for now


class PointGreyDetector6BMB(SingleTrigger, AreaDetector):
    """"""
    cam = ADComponent(MyPointGreyDetectorCam, "cam1:")
    
    tiff1 = ADComponent(
        TIFFPlugin,
        suffix="TIFF1:",
        root="/",                               # for databroker
        write_path_template=FILE_PATH,          # for EPICS AD
    )
    
    proc1 = ADComponent(
        ProcessPlugin,
        suffix="PROC1",
    )


In [10]:
# query APS status

aps = APS_devices.ApsMachineParametersDevice(name="APS")


In [ ]:
# TODO: operation status checker for 6bm

def operations_in_6bmb():
    pass

Need additional setup IOC (soft) for this status.
recommended method `put in VME`

```python
instrument_in_use = EpicsSignalRO(
    "2bm:instrument_in_use", 
    name="instrument_in_use")

def operations_in_2bmb():
    """returns True if allowed to use X-ray beam in 2-BM-B station"""
    v = instrument_in_use.value
    enums = instrument_in_use.enum_strs
    return enums[v] == "2-BM-B"
```

In [12]:
aps.inUserOperations

TimeoutError: S:DesiredMode could not connect within 1.0-second timeout.

# Configur motors

In [21]:
from ophyd import MotorBundle

class TomoStage(MotorBundle):
    #rotation
    preci = Component(EpicsMotor, "6bmpreci:m1",
                      name='preci',
                     )
    
    samX = Component(EpicsMotor, "???",
                     name='samX',
                    )
    
    samY = Component(EpicsMotor, "???",
                     name="samY",
                    )

tomostage = TomoStage()

samx = tomostage.samX
samy = tomostage.samY
samr = tomostage.preci

TypeError: __init__() missing 1 required keyword-only argument: 'name'

In [ ]:
import ophyd.sim


# UserCalc

In [ ]:
# grab all usercalcs
import apstools.synApps_ophyd
calcs = apstools.synApps_ophyd.userCalcsDevice("6bma1:", name="calcs")

# 2 is ambient light checker
calc2 = clacs.calc2


# Scan plans

In [19]:
from bluesky import RunEngine
import bluesky.plans as bp
import bluesky.plan_stubs as bps
from bluesky.callbacks.best_effort import BestEffortCallback

ModuleNotFoundError: No module named 'matplotlib'

In [20]:
RE = RunEngine({})
RE.subscribe(BestEffortCallback())

NameError: name 'BestEffortCallback' is not defined

In [ ]:
det = PointGreyDetector6BMB()

n_white = 10
n_dark = 10

samOutDist = -4.00  # mm

omega_step = 0.25                            # degrees
number_of_projections = int(180/omega_step)  # 0 ~ 180

# area detector stage config paras
trigger_wait = 0.01  # sec
detector_gain = 5    # sec

exposure_time = 


# setting monitor
# NOTE:
#    part of the scan plan
if operations_in_6bma() and aps.inUserOperations:
    # no scans until A_shutter is open
    suspend_A_shutter = bluesky.suspenders.SuspendFloor(
        A_shutter.pss_state, 1)
    #suspend_A_shutter.install(RE)
    RE.install_suspender(suspend_A_shutter)
    
     # no scans if aps.current is too low
    suspend_APS_current = bluesky.suspenders.SuspendFloor(
        aps_current, 2, resume_thresh=10)
    RE.install_suspender(suspend_APS_current)
    

def collect_white_field():
    
    current_samx = samx.position
    
    yield from bps.mv(
        A_shutter, "open", 
        samx, current_samx+samOutDist,
        det.cam.trigger_mode, "Internal",
        det.cam.num_images, n_white,
    )
    
    yield from bps.trigger(det)
    
    yield from lambda _: while det.tiff1.num_captured.value < n_white: yield from bps.sleep(0.01)

    yield from bps.mv(
        samx, current_samx
    )


def collect_white_field():
    
    current_samx = samx.position
    
    yield from bps.mv(
        A_shutter, "close", 
        samx, current_samx+samOutDist,
        det.cam.trigger_mode, "Internal",
        det.cam.num_images, n_dark,
    )
    
    yield from bps.trigger(det)
    
    yield from lambda _: while det.tiff1.num_captured.value < n_white: yield from bps.sleep(0.01)

    yield from bps.mv(
        samx, current_samx
    )
    
    
def collect_projection():
    
    yield from bp.scan(
        [det], 
        samr, 0, 180, number_of_projections,
    )



In [ ]:
# check plan

summarize